In [29]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [30]:
# Variables

TOTAL_IMAGES = 10  # total unique: 4320

all_images = []

background = ["blue", "gold", "green", "peach", "pink"]
background_weights = [10, 5, 15, 40, 30]

color = ["black", "blue", "deepblue", "green", "orange", "purple", "red", "white", "yellow"]
color_weights = [2, 5, 9, 12, 20, 15, 27, 3, 7]

spots = ["black", "ice", "white"]
spots_weights = [20, 3, 78]

environment = ["field", "swamp", "desert", "arctic"]
environment_weights = [50, 30, 15, 5]

toxicity = ["healthy", "toxic"]
toxicity_weights = [85, 15]

species = ["toadstool", "mushrooms"]
species_weights = [80, 20]

magic_weights = [5, 95]
magic = ["magic", "ordinary"]


In [31]:
def create_new_image():
    new_image = {}

    # For each trait category, select a random trait based on the weightings
    new_image["Background"] = random.choices(background, background_weights)[0]
    new_image["Color"] = random.choices(color, color_weights)[0]
    new_image["Spots"] = random.choices(spots, spots_weights)[0]
    new_image["Environment"] = random.choices(environment, environment_weights)[0]
    new_image["Toxicity"] = random.choices(toxicity, toxicity_weights)[0]
    new_image["Species"] = random.choices(species, species_weights)[0]
    new_image["Magic"] = random.choices(magic, magic_weights)[0]

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image


for i in range(TOTAL_IMAGES):
    new_image = create_new_image()

    all_images.append(new_image)


In [32]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [33]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [34]:
# print(all_images)

In [35]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0

color_count = {}
for item in color:
    color_count[item] = 0

spots_count = {}
for item in spots:
    spots_count[item] = 0

environment_count = {}
for item in environment:
    environment_count[item] = 0

toxicity_count = {}
for item in toxicity:
    toxicity_count[item] = 0

species_count = {}
for item in species:
    species_count[item] = 0

magic_count = {}
for item in magic:
    magic_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    color_count[image["Color"]] += 1
    spots_count[image["Spots"]] += 1
    environment_count[image["Environment"]] += 1
    toxicity_count[image["Toxicity"]] += 1
    species_count[image["Species"]] += 1
    magic_count[image["Magic"]] += 1

print(background_count)
print(color_count)
print(spots_count)
print(environment_count)
print(toxicity_count)
print(species_count)
print(magic_count)


{'blue': 52, 'gold': 47, 'green': 71, 'peach': 144, 'pink': 106}
{'black': 6, 'blue': 21, 'deepblue': 32, 'green': 52, 'orange': 77, 'purple': 66, 'red': 101, 'white': 15, 'yellow': 50}
{'black': 126, 'ice': 43, 'white': 251}
{'field': 173, 'swamp': 137, 'desert': 71, 'arctic': 39}
{'safe': 312, 'toxic': 108}
{'toadstool': 337, 'mushrooms': 83}
{'true': 47, 'false': 373}


In [36]:
# Generate Metadata for all Traits
METADATA_FILE_NAME = './metadata/all-traits.json'
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

print("Created all-traits JSON file")


Created all-traits JSON file


In [37]:
# Generate Images
for item in all_images:

    # print("sourcing images for", item)

    # source all images
    # BASE IMAGES
    bgImage = Image.open(
        f'./trait-layers/backgrounds/{item["Background"]}.png').convert('RGBA')
    environmentImage = Image.open(
        f'./trait-layers/environments/{item["Environment"]}.png').convert('RGBA')
    toadstoolColor = Image.open(
        f'./trait-layers/colors/{item["Color"]}-toadstool.png').convert('RGBA')
    toadstoolOutline = Image.open(
        f'./trait-layers/general/outline-toadstool.png').convert('RGBA')

    # CONDITIONALLY IMPORT IMAGES    
    if item["Species"] == "mushrooms":
        mushroomsColor = Image.open(
            f'./trait-layers/colors/{item["Color"]}-mushrooms.png').convert('RGBA')
        mushroomsOutline = Image.open(
            f'./trait-layers/general/outline-mushrooms.png').convert('RGBA')
        shadingImage = Image.open(
            f'./trait-layers/general/shading-mushrooms.png').convert('RGBA')
    else:    
        shadingImage = Image.open(
            f'./trait-layers/general/shading-toadstool.png').convert('RGBA')
        toadstoolSpots = Image.open(
            f'./trait-layers/spots/{item["Spots"]}.png').convert('RGBA')

    # properties to change only when item is of toxic variety
    if item["Toxicity"] == "toxic":
        toxicImage = Image.open(
            f'./trait-layers/general/toxic.png').convert('RGBA')
        # change the alpha channels of the images for giving the toxic-look
        toxicImage.putalpha(50)
        shadingImage.putalpha(144)

    # START to layer all the images together
    tsColWithBg = Image.alpha_composite(bgImage, toadstoolColor)
    toadstoolBaseImage = Image.alpha_composite(tsColWithBg, toadstoolOutline)

    # create the mushrooms base (outline & color) if required
    if item["Species"] == "mushrooms":
        mushroomsBaseImage = Image.alpha_composite(mushroomsColor, mushroomsOutline)
        preshadedImage = Image.alpha_composite(toadstoolBaseImage, mushroomsBaseImage)
    else:
        preshadedImage = Image.alpha_composite(toadstoolBaseImage, toadstoolSpots)
    
    # add the spores to magic items
    if item["Magic"] == "magic":
        sporesImage = Image.open(f'./trait-layers/spores/{item["Color"]}.png').convert('RGBA')
        preshadedImage = Image.alpha_composite(preshadedImage, sporesImage)

    # add the shading
    shadedImage = Image.alpha_composite(preshadedImage, shadingImage)

    # add the environment
    finalImage = Image.alpha_composite(shadedImage, environmentImage)

    # add toxic odour lines (if toxic)
    if item["Toxicity"] == "toxic":
        finalImage = Image.alpha_composite(finalImage, toxicImage)

    # Convert to RGB
    rgb_im = finalImage.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)


In [38]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"].title()))
    token["attributes"].append(getAttribute("Color", i["Color"].title()))
    token["attributes"].append(getAttribute("Spots", i["Spots"].title()))
    token["attributes"].append(getAttribute("Environment", i["Environment"].title()))
    token["attributes"].append(getAttribute("Toxicity", i["Toxicity"].title()))
    token["attributes"].append(getAttribute("Species", i["Species"].title()))
    token["attributes"].append(getAttribute("Magic", i["Magic"].title()))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()